In [ ]:
# !pip install influxdb-client --quiet

import pandas as pd
from influxdb_client import InfluxDBClient
from influxdb_client.client.warnings import MissingPivotFunction
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

warnings.simplefilter("ignore", MissingPivotFunction)

# === InfluxDB Setup ===
url = 'http://192.52.36.237:8086'
token = 'iwht7fARF0X2HrqAez_bLIomiTa5Umj2JmX0wEJeVs5TXgSy0w-NsXx1TyxlGCHg77Icnk4x0Qf8yAYY_WYMsQ=='
org = 'bodensonde'
bucket = 'sensor_data'
client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

# === Load signal data ===
# signal_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/imtek/signal_data_all_gateways.csv", parse_dates=["message_received_at"])
# signal_df["timestamp"] = signal_df["message_received_at"].dt.tz_convert(None)
# start = signal_df["timestamp"].min().isoformat() + "Z"
# end = signal_df["timestamp"].max().isoformat() + "Z"


# start = "2025-06-15T00:00:00Z"
# end = "2025-06-22T00:00:00Z"

# === Query helper for each node
def query_field(field, device, rename=None):
    query = f'''
from(bucket: "sensor_data")
  |>  range(start: 2025-04-01T00:00:00Z, stop: 2025-06-15T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "environmental_data")
  |> filter(fn: (r) =>  r["_field"] == "supercap_voltage" or r["_field"] == "bus_voltage")
  |> filter(fn: (r) => r["device_id"] == "bodensonde-test-node-3")
  |> keep(columns: ["_time", "_value", "_value"])  
  |> sort(columns: ["_value"], desc: false)
  
  
'''
    print(query)
    df = query_api.query_data_frame(query)
    if df.empty or "_time" not in df or "_value" not in df:
        return pd.DataFrame(columns=["Time", rename or field])
    df = df[["_time", "_value"]].rename(columns={"_time": "Time", "_value": rename or field})
    df["Time"] = pd.to_datetime(df["Time"], errors = "coerce").dt.tz_convert(None)
    return df

# === Processing logic per node
def process_node(device_name, node_label):
    df_temp = query_field("air_temperature", device=device_name)
    df_hum = query_field("air_humidity", device=device_name)
    df_supercap = query_field("supercap_voltage", device=device_name)
    df_bus_voltage = query_field("bus_voltage", device=device_name)
    df_current = query_field("current", device=device_name, rename="current")
    # df_curr = query_field("current", device=device_name, rename="sun_light")
    # print(df_curr)
    #print(df_temp)
    if df_supercap.empty or df_bus_voltage.empty :
        print(f"Missing data for {device_name}")
        return
    
    df_combined = pd.merge_asof(
        # df_temp.sort_values("Time"),
        # df_hum.sort_values("Time"),
        df_supercap.sort_values("Time"),
        df_bus_voltage.sort_values("Time"),
        # df_current.sort_values("Time"),
        on="Time",
        direction="nearest",
        tolerance=pd.Timedelta("5min")
    )

    #df_combined = df_combined.sort_values("Time")
    df_combined.to_csv(f"{device_name}_all.csv", index=False)
    print(f"Saved to {device_name}_all.csv")



    # if df_curr.empty:
    #     print(f"No sun_light data found for device: {device_name}")
        
    #     return
    # df_curr = df_curr.dropna(subset=["Time"])
    # df_curr = df_curr[(df_curr["Time"].dt.hour >= 6) & (df_curr["Time"].dt.hour <= 20)]

    # Merge probe data
    # probe_df = pd.merge_asof(
    #     pd.merge_asof(df_temp.sort_values("Time"), df_hum.sort_values("Time"), on="Time", direction="nearest"),
    #     df_curr.sort_values("Time"), on="Time", direction="nearest"
    # )

    # # Match signal data
    # signal_node = signal_df[signal_df["device_id"].str.contains(device_name, case=False)].sort_values("timestamp")
    # probe_df = probe_df.sort_values("Time")
    # combined = pd.merge_asof(
    #     signal_node, probe_df, left_on="timestamp", right_on="Time",
    #     direction="nearest", tolerance=pd.Timedelta("15m")
    # )

    # # Correlation
    # clean = combined[["rssi", "snr", "air_temperature", "air_humidity", "sun_light"]].dropna()
    # corr = clean.corr()

        # Save air temperature time series to CSV
    # df_temp = df_temp.sort_values("Time")

    # df_temp.to_csv(f"{device_name}_air_temperature.csv", index=False)
    # print(f"Saved time series to {device_name}_air_temperature.csv")


    

# === Run for both nodes
# process_node("bodensonde-test-node-2", "Test Node 2")
process_node("bodensonde-test-node-3", "Test Node 3")



from(bucket: "sensor_data")
  |>  range(start: 2025-04-01T00:00:00Z, stop: 2025-06-15T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "environmental_data")
  |> filter(fn: (r) => r["_field"] == "air_humidity" or r["_field"] == "air_temperature" or r["_field"] == "supercap_voltage" or r["_field"] == "bus_voltage" or r["_field"] == "current")
  |> filter(fn: (r) => r["device_id"] == "bodensonde-test-node-3")
  |> keep(columns: ["_time", "_value", "_value", "_value", "_value", "_value"])  
  |> sort(columns: ["_value"], desc: false)




from(bucket: "sensor_data")
  |>  range(start: 2025-04-01T00:00:00Z, stop: 2025-06-15T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "environmental_data")
  |> filter(fn: (r) => r["_field"] == "air_humidity" or r["_field"] == "air_temperature" or r["_field"] == "supercap_voltage" or r["_field"] == "bus_voltage" or r["_field"] == "current")
  |> filter(fn: (r) => r["device_id"] == "bodensonde-test-node-3")
  |> keep(columns: ["_time", "_valu